In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [5]:
device = torch.device("mps")

In [122]:
df = pd.read_parquet('bal_train.parquet')
df = df[~df.isnull()]
df = df[df['audio_embedding'].apply(lambda x: len(x)) == 10]
print(df.shape)

embeddings = np.vstack(df['audio_embedding'].apply(lambda x: np.vstack(x))).reshape(-1,10,128)
embedding_tensor = torch.Tensor(embeddings)
print(embedding_tensor.size())

(21782, 4)
torch.Size([21782, 10, 128])


In [137]:
def label_converter(x):
    output = np.zeros(527,dtype=int)
    for label in x:
        output[label] = 1
    return output

df['label'] = df['labels'].apply(lambda x: label_converter(x))

labels = np.vstack(df['label']).reshape(-1,527)
label_tensor = torch.Tensor(labels)
print(label_tensor.size())

torch.Size([21782, 527])


In [139]:
train_set = TensorDataset(embedding_tensor, label_tensor)

In [ ]:
bs = 16
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=bs, shuffle=True)